# Instalação do AutoGluon

Para o notebook, é necessário instalar uma biblioteca chamada AutoGluon. Já alertamos que é uma biblioteca pesada e que roda com um consumo considerável de memória.

In [1]:
%pip install autogluon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using ca

# Setup

In [1]:
import pandas as pd
import os
import numpy as np
import sklearn
import sklearn.metrics

from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from autogluon.tabular import TabularPredictor
from autogluon.core.metrics import make_scorer

# Caminhos para leitura e escrita dos arquivos

A fim de disponibilizar aos modelos os datasets corretos, definimos como caminho para a leitura os arquivos CSV gerados na etapa de análise exploratória e limpeza, os quais estão disponíveis no diretório `data`. Além disso o arquivo de predição, que é submetido no Kaggle, chama-se `predictions_autogluon.csv`

In [14]:
train_file_path = '../data/bank_train.csv'
test_file_path = '../data/bank_test.csv'
output_file_path = '../data/predictions_autogluon.csv'

output_dir = os.path.dirname(output_file_path)
os.makedirs(output_dir, exist_ok=True)

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Pré-processamento dos dados

Durante a análise exploratória, foram identificados diversos aspectos importantes, como a presença de várias features categóricas e numéricas, outliers questionáveis que poderiam prejudicar o desempenho do modelo, além de uma grande porcentagem de respostas "no" na feature target, o que poderia influenciar o modelo a prever que o serviço bancário não seria contratado.

Por essa razão, optamos por realizar o pré-processamento nesta etapa em vez de durante a análise exploratória, pois não sabíamos como os modelos reagiriam. As ações realizadas foram as seguintes:

- **Codificação das features categóricas**: Utilizamos o OneHotEncoder, cujo princípio de funcionamento consiste em criar colunas no dataset com valores 0 ou 1 para as categorias da feature. Por exemplo, na feature `marital`, que possui 3 opções, seriam criadas 3 colunas, e se a pessoa for casada, será atribuído 1 na coluna `married` e 0 nas outras duas colunas criadas.

- **Balanceamento da feature target**: Testamos algumas abordagens, como o uso do `SMOTE` para criar novas amostras da classe minoritária ("yes"), o que resultou em overfitting. Outra opção foi considerar uma pequena porcentagem de "no", de modo a obter uma proporção aceitável entre as duas classes. Para isso, utilizamos o método `sample()`, que seleciona aleatoriamente uma porcentagem das linhas da classe majoritária.

- **Normalização dos dados**: Aplicamos o `RobustScaler`, uma ferramenta que transforma os dados para que os valores sejam escalados com base na mediana e no intervalo interquartil (IQR), em vez da média e do desvio padrão, como é feito pelo `StandardScaler`. Esse método é menos sensível a outliers.

- **Exclusão de algumas features**: certas features foram excluídas, como a *id*, que não possui um significado definido, e a *poutcome*, que contém mais dados `unknown` do que dados significativos.


In [15]:
train_ids = train_data['id']
test_ids = test_data['id']

train_data = train_data.drop(columns=['id', 'poutcome'])
test_data = test_data.drop(columns=['id', 'poutcome'])

label = 'y'

categorical_features = train_data.select_dtypes(include=['object']).columns

# Pipeline para pré-processamento das colunas categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Balancear classes removendo linhas da classe majoritária
majority_class = train_data[train_data[label] == 0]
minority_class = train_data[train_data[label] == 1]

# Manter 19% da classe majoritária (no)
majority_class_sampled = majority_class.sample(frac=0.18, random_state=42)

train_data_balanced = pd.concat([majority_class_sampled, minority_class])

X = train_data_balanced.drop(columns=[label])
y = train_data_balanced[label]

X_processed = preprocessor.fit_transform(X)

# Escalar os dados numéricos
scaler = RobustScaler()
X_processed = scaler.fit_transform(X_processed)

train_data_balanced_processed = pd.DataFrame(X_processed)
train_data_balanced_processed[label] = y.reset_index(drop=True)

# AutoGluon: Automatização de Modelagem Preditiva

AutoGluon é uma biblioteca de machine learning que facilita a criação e otimização de modelos preditivos de alta qualidade, sem a necessidade de extensa expertise técnica em ajuste de hiperparâmetros ou seleção de modelos. Ele é especialmente útil para tarefas como classificação, regressão, previsão de séries temporais e processamento de imagens e textos. AutoGluon automatiza o processo de treinamento, validação e seleção do melhor modelo, permitindo que os usuários alcancem bons resultados com o mínimo de intervenção manual.

## Por que utilizamos o AutoGluon?

1. **Facilidade de Uso**: AutoGluon é projetado para ser simples de utilizar, permitindo que até usuários com pouca experiência em machine learning possam treinar modelos eficazes. Ele automatiza tarefas complexas, como a engenharia de features, ajuste de hiperparâmetros, e até mesmo a escolha do modelo mais adequado para o conjunto de dados.

2. **Eficiência**: A biblioteca realiza busca automatizada de hiperparâmetros e empilhamento (stacking) de modelos, resultando em predições de alta qualidade. Além disso, a biblioteca pode balancear o tempo de execução com a performance do modelo, adaptando-se às necessidades específicas do projeto.

3. **Personalização**: Apesar de ser altamente automatizado, AutoGluon permite ajustes personalizados, como a definição de métricas específicas de avaliação, o que torna a ferramenta flexível e adaptável a diferentes cenários de modelagem.

No código abaixo, configuramos o AutoGluon para utilizar a métrica `roc_auc_score` como critério de avaliação.

In [16]:
ag_roc_auc_scorer = make_scorer(name='roc_auc_score',
                                 score_func=sklearn.metrics.roc_auc_score,
                                 optimum=1,
                                 greater_is_better=True)

predictor = TabularPredictor(label=label, eval_metric=ag_roc_auc_scorer, sample_weight='auto_weight').fit(
    train_data_balanced_processed, presets='best_quality'
)


No path specified. Models will be saved in: "AutogluonModels/ag-20240805_232450"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.83 GB / 12.67 GB (77.5%)
Disk Space Avail:   71.12 GB / 107.72 GB (66.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be 

# Pré-processamento do dataset de teste e escrita das predições

In [19]:
X_test_processed = preprocessor.transform(test_data)
X_test_processed = scaler.transform(X_test_processed)

predictions = predictor.predict(pd.DataFrame(X_test_processed))

test_data['y'] = predictions

# Adicionar a coluna 'id' de volta ao dataset de teste
test_data['id'] = test_ids

test_data[['id', 'y']].to_csv(output_file_path, index=False)

# LeaderBoard

O AutoGluon permite visualizar, em ranking, os modelos testados com as características de seus respectivos treinamentos.

In [18]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.880153,roc_auc_score,18.018905,2082.077194,0.005867,3.780497,3,True,32
1,XGBoost_BAG_L2,0.879423,roc_auc_score,12.410753,1769.670652,0.535367,72.272238,2,True,28
2,LightGBM_BAG_L2,0.879303,roc_auc_score,12.925318,1767.700969,1.049932,70.302555,2,True,21
3,LightGBMLarge_BAG_L2,0.878205,roc_auc_score,13.493980,1817.134804,1.618594,119.736390,2,True,30
4,LightGBMXT_BAG_L2,0.877628,roc_auc_score,12.088119,1750.456640,0.212733,53.058226,2,True,20
5,NeuralNetTorch_BAG_L2,0.877523,roc_auc_score,12.217551,1920.294954,0.342165,222.896540,2,True,29
6,RandomForestEntr_BAG_L2,0.876784,roc_auc_score,12.738679,1712.632197,0.863293,15.233783,2,True,23
7,RandomForestGini_BAG_L2,0.875584,roc_auc_score,12.711687,1711.956446,0.836301,14.558032,2,True,22
8,CatBoost_BAG_L2,0.875273,roc_auc_score,11.960737,1771.310513,0.085351,73.912098,2,True,24
9,CatBoost_r177_BAG_L2,0.874588,roc_auc_score,12.028185,1732.240167,0.152799,34.841753,2,True,31
